#Dataset

Usando como referencia a documentação de Ponto de Partida

In [1]:
!pip install kaggle
!pip install keras-tuner -q
!kaggle datasets download -d arbazkhan971/cuhk-face-sketch-database-cufs --force
!unzip -oq "cuhk-face-sketch-database-cufs.zip"
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.7 MB/s eta 0:00:00
Dataset URL: https://www.kaggle.com/datasets/arbazkhan971/cuhk-face-sketch-database-cufs
License(s): copyright-authors
 99% 112M/113M [00:04<00:00, 32.7MB/s]
100% 113M/113M [00:04<00:00, 26.3MB/s]


# Importando Bibliotecas

In [2]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image as PILImage
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score,roc_curve,roc_auc_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import display, Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from IPython.display import display
import pandas as pd
from torchvision import transforms
from tensorflow.keras import backend as K
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

# Funções

In [3]:
def list_files_in_folder(folder_path): #Retorna a lista com o nome dos arquivos na pasta.
    try:
      file_list = os.listdir(folder_path)
      return file_list
    except FileNotFoundError:
      print(f"Error: Folder not found at {folder_path}")
      return []

def display_images_from_folder(folder_path): #exibe todas as imagens da pasta
    image_files = list_files_in_folder(folder_path)
    if not image_files:
      print(f"No images found in folder: {folder_path}")
      return

    for image_file in image_files:
      image_path = os.path.join(folder_path, image_file)
      print(f"Exibindo: {image_file}")
      display(Image(filename=image_path))

def label_sex(folder_path): # Retorna uma lista de rótulos 0 ou 1 baseado
                  # no nome do arquivo. Verifiquei todos os
                  # arquivos para garantir que seguem esse padrão.
      labels = []
      image_files = list_files_in_folder(folder_path)
      for file in image_files:
          if file.lower().startswith('m'):
            labels.append(0)
          else:
            labels.append(1)
            return labels

def norm_image(folder_path):
        normalized_images = []
        image_files = list_files_in_folder(folder_path)
        for file in image_files:
          image_path = os.path.join(folder_path, file)
          image = PILImage.open(image_path)
          image_array = np.array(image) / 255.0
          normalized_images.append(image_array)
        return normalized_images

def graying_image(folder_path): #Função para criar a pasta que armazena as imagens em preto e branco.
  input_folder = folder_path
  output_folder = "photos_gray"
  os.makedirs(output_folder, exist_ok=True)

  for file in list_files_in_folder(folder_path):
    image_path = os.path.join(input_folder, file)
    with PILImage.open(image_path) as image:
      gray_image = image.convert("L")
      gray_image.save(os.path.join(output_folder, file))

def best_hp(hp, color_scale=3, nome="TESTE"):

        model = models.Sequential(name=nome)
        hp_filters = hp.Choice('filters', values=[31, 62, 129])
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])
        hp_activation = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'softmax'])

        model.add(layers.Input(shape=(250, 200, color_scale)))

        model.add(layers.Conv2D(hp_filters, (6, 6), activation=hp_activation, padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Conv2D(hp_filters * 2, (5, 5), activation=hp_activation, padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Conv2D(hp_filters * 3, (4, 4), activation=hp_activation, padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Conv2D(hp_filters * 4, (3, 3), activation=hp_activation, padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        optimizer_custom = Adam(learning_rate=hp_learning_rate)
        model.compile(optimizer=optimizer_custom,
                loss='binary_crossentropy',
                metrics=['accuracy'])
        return model

def top_melhores_cnn_da_residencia(color_scale,nome,hp_activation,hp_filters,lr):
    model = models.Sequential(name=nome)
    model.add(layers.Input(shape=(250, 200, color_scale)))

    model.add(layers.Conv2D(hp_filters, (6, 6), activation=hp_activation, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(hp_filters * 2, (5, 5), activation=hp_activation, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(hp_filters * 3, (4, 4), activation=hp_activation, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(hp_filters * 4, (3, 3), activation=hp_activation, padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))



    model.add( tf.keras.layers.Flatten() )
    model.add( tf.keras.layers.Dense(1, activation='sigmoid') )
    optimizer_custom = Adam(learning_rate=lr)
    model.compile(optimizer_custom,
              loss=['binary_crossentropy'],
              metrics=(['accuracy']))
    return model

# Treinamento

##Chamar a pasta com as fotos

In [15]:
photos_folder = "photos"
files_in_photos = list_files_in_folder(photos_folder)

##Modelo com as imagens originais provenientes do dataset

In [16]:
labels = label_sex(photos_folder)

In [18]:
normalized_images = norm_image(photos_folder)

In [21]:
X = normalized_images  #Array de imagens normalizadas.
y = labels
X = np.expand_dims(X, axis=-1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, stratify=y, random_state=23)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, stratify=y_temp, random_state=23)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

ValueError: Found input variables with inconsistent numbers of samples: [188, 1]

In [ ]:
tuner = kt.Hyperband(
    best_hp,
    objective=kt.Objective("accuracy", direction="max"),
    max_epochs=20,
    factor=3,
    directory='model_results_2',
    project_name='grid_search'
)

In [ ]:
tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Melhores hiperparâmetros encontrados: {best_hps.values}")

In [ ]:
activation = 'relu'
filters = 62
lr = 0.01
model = top_melhores_cnn_da_residencia(3,'M-1678',activation,filters,lr)
model.summary()

In [ ]:
early_stopping = EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)
model.fit(X_train,
          y_train,
          batch_size=16,
          epochs=20,
          verbose=1,
          callbacks=[early_stopping],)

In [ ]:
model.save('face_model.keras')

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Loss na validação: {val_loss}")
print(f"Acurácia na validação: {val_accuracy}")

In [ ]:
pred = model.predict(X_val)
pred_labels = (pred > 0.5).astype(int)

In [ ]:
auc = roc_auc_score(y_val, pred_labels)
print(f"AUC-ROC: {auc}")
f1 = f1_score(y_val, pred_labels)
print(f"F1-Score: {f1}")
fpr, tpr, thresholds = roc_curve(y_val, pred_labels)

# Plotando a curva ROC
plt.plot(fpr, tpr, marker='.', label='Modelo')
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC')
plt.legend()
plt.show()

for i in range(len(X_val)):
  if(y_val[i] !=pred_labels[i][0]):#Analisando as fotos em que o modelo errou a
                                   #previsão
      plt.imshow(X_val[i].reshape(250, 200,3), cmap=None)
      plt.title(f"Verdadeiro: {y_val[i]} | Previsto: {pred_labels[i][0]}")
      plt.axis('off')
      plt.show()

##Modelo com imagens convertidas para preto e branco

Para criar uma rede capaz de diferenciar rostos masculinos de femininos, transformamos todas as imagens para preto e branco, pois a cor não é um fator determinante para essa característica. No entanto, também treinamos o modelo com as imagens originais para verificar se há alguma melhora no desempenho.

In [ ]:
graying_image(photos_folder)
photos_gray_folder = "photos_gray"

In [ ]:
display_images_from_folder(photos_gray_folder)

In [ ]:
labels = label_sex(photos_gray_folder)

In [ ]:
print(f"Quantidade de 0s: {labels.count(0)}")
print(f"Quantidade de 1s: {labels.count(1)}")

In [ ]:
normalized_gray_images = norm_image(photos_gray_folder)

In [ ]:
X = normalized_gray_images
y = labels
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.5, stratify=y, random_state=23)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.4, stratify=y_temp, random_state=23)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
model_gray = top_melhores_cnn_da_residencia(1,'G-0909',activation,filters,lr)
model_gray.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
early_stopping = EarlyStopping(
    monitor='loss',   # Métrica a ser monitorada
    patience=5,           # Número de épocas sem melhoria antes de parar
    restore_best_weights=True  # Restaura os pesos da melhor época
)
model_gray.fit(X_train,
          y_train,
          batch_size=16,
          epochs=20,
          verbose=1,
          callbacks=[early_stopping])

In [ ]:
model_gray.save('gray_face_model.keras')

In [ ]:
val_loss, val_accuracy_gray = model_gray.evaluate(X_val, y_val, verbose=1)
print(f"Loss na validação: {val_loss}")
print(f"Acurácia na validação: {val_accuracy_gray}")

In [ ]:
pred = model_gray.predict(X_val)
pred_labels = (pred > 0.5).astype(int)

In [ ]:
auc = roc_auc_score(y_val, pred_labels)
print(f"AUC-ROC: {auc}")
f1 = f1_score(y_val, pred_labels)
print(f"F1-Score: {f1}")
fpr, tpr, thresholds = roc_curve(y_val, pred_labels)

plt.plot(fpr, tpr, marker='.', label='Modelo')
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC')
plt.legend()
plt.show()

for i in range(len(X_val)):
  if(y_val[i] != pred_labels[i][0]):
      plt.imshow(X_val[i].reshape(250, 200,1), cmap="gray")
      plt.title(f"Verdadeiro: {y_val[i]} | Previsto: {pred_labels[i][0]}")
      plt.axis('off')
      plt.show()

## Modelo com data augmentation

In [ ]:
labels = label_sex(photos_folder)

In [ ]:
normalized_images = norm_image(photos_folder)

In [ ]:
X = normalized_images
y = labels
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.5, stratify=y, random_state=23)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.4, stratify=y_temp, random_state=23)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)
train_generator = datagen.flow(
    X_train, y_train, batch_size=16
)

In [ ]:
tuner = kt.Hyperband(
    best_hp,
    objective=kt.Objective("accuracy", direction="max"),
    max_epochs=20,
    factor=3,
    directory='model_results_3',
    project_name='grid_search_data_ag'
)

In [ ]:
tuner.search(train_generator, epochs=20, validation_data=(X_test, y_test))

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Melhores hiperparâmetros encontrados: {best_hps.values}")

In [ ]:
activation = 'relu'
filters = 31
lr = 0.001
model = top_melhores_cnn_da_residencia(3,'DA-1882',activation,filters,lr)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
early_stopping = EarlyStopping(
    monitor='loss',   # Métrica a ser monitorada
    patience=9,           # Número de épocas sem melhoria antes de parar
    restore_best_weights=True  # Restaura os pesos da melhor época
)

In [ ]:
history = model.fit(train_generator,#usar searchgrig hyperopt
          batch_size=16,
          epochs=50,
          verbose=1,
          callbacks=[early_stopping])

In [ ]:
model.save('face_model.keras')

In [ ]:
val_loss, val_accuracy= model.evaluate(X_val, y_val, verbose=1)
print(f"Loss na validação: {val_loss}")
print(f"Acurácia na validação: {val_accuracy_data_ag}")

In [ ]:
pred = model.predict(X_val)
pred_labels = (pred > 0.5).astype(int)

In [ ]:
auc = roc_auc_score(y_val, pred_labels)
print(f"AUC-ROC: {auc}")
f1 = f1_score(y_val, pred_labels)
print(f"F1-Score: {f1}")
fpr, tpr, thresholds = roc_curve(y_val, pred_labels)

plt.plot(fpr, tpr, marker='.', label='Modelo')
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC')
plt.legend()
plt.show()

for i in range(len(X_val)):
  if(y_val[i] !=pred_labels[i][0]):
      plt.imshow(X_val[i].reshape(250, 200,3), cmap=None)
      plt.title(f"Verdadeiro: {y_val[i]} | Previsto: {pred_labels[i][0]}")
      plt.axis('off')
      plt.show()